In [1]:
import ollama
import pandas as pd
from rich import print as p
from sklearn.metrics import classification_report

# Results I

In [2]:
datasets = ('facebook', 'twitter')
models = ('llama3', 'mistral', 'gemma2', 'gpt-3.5-turbo-0125')

In [3]:
for data in datasets:
    df = pd.read_csv(f'../data/opinions/{data}_cachaca.csv')
    for model in models:
        y_true, y_pred = df['labels'], df[model]
        p(f'[bold red]{data}: {model}[/bold red]')
        p(classification_report(y_true, y_pred))

facebook: llama3

precision    recall  f1-score   support

    negative       0.12      0.87      0.22        31
     neutral       0.78      0.67      0.72       590
    positive       0.83      0.67      0.74       578

    accuracy                           0.68      1199
   macro avg       0.58      0.74      0.56      1199
weighted avg       0.78      0.68      0.72      1199

facebook: mistral

precision    recall  f1-score   support

    negative       0.21      0.74      0.33        31
     neutral       0.79      0.70      0.74       590
    positive       0.76      0.74      0.75       578

    accuracy                           0.72      1199
   macro avg       0.59      0.73      0.61      1199
weighted avg       0.76      0.72      0.74      1199

facebook: gemma2

...

precision    recall  f1-score   support

    negative       0.17      0.68      0.27        19
     neutral       0.98      0.26      0.40       878
    positive       0.13      0.89      0.23       103

    accuracy                           0.33      1000
   macro avg       0.43      0.61      0.30      1000
weighted avg       0.88      0.33      0.38      1000

twitter: gemma2

precision    recall  f1-score   support

    negative       0.26      0.53      0.35        19
     neutral       0.97      0.34      0.50       878
    positive       0.15      0.95      0.26       103

    accuracy                           0.41      1000
   macro avg       0.46      0.61      0.37      1000
weighted avg       0.88      0.41      0.48      1000

twitter: gpt-3.5-turbo-0125

precision    recall  f1-score   support

    negative       0.23      0.79      0.36        19
     neutral       0.96      0.38      0.54       878
    positive       0.14      0.81      0.24       103

    accuracy                           0.43      1000
   macro avg       0.44      0.66      0.38      1000
weighted avg       0.86      0.43      0.51      1000

# Evaluating manually labeled data

I will evaluate whether the data are opinion messages or not. Example:

The sentence "This is the new Tom Cruise movie" is not an opinion sentence. Whereas, the sentence "This new Tom Cruise movie is amazing!" is an opinion message.

To do this, I will use the knowledge of the LLM to perform this evaluation.

A message that is not an opinion is different from a neutral message. A non-opinion message simply states a fact or provides information without expressing a personal judgment or sentiment. On the other hand, a neutral message may contain subjective elements but does not convey a strong positive or negative sentiment. For instance, "This movie was released in 2023" is a non-opinion message, while "The movie is okay" is a neutral message.

In [4]:
PROMPT_TEMPLATE = """
Avalie a seguinte mensagem para determinar se ela expressa uma opinião. 
Uma mensagem que não é uma opinião é diferente de uma mensagem neutra. 
Uma mensagem que não é uma opinião simplesmente afirma um fato ou fornece 
informação sem expressar um julgamento pessoal ou sentimento. Por outro lado, 
uma mensagem neutra pode conter elementos subjetivos, mas não transmite um 
sentimento fortemente positivo ou negativo. Por exemplo, "Este filme foi 
lançado em 2023" é uma mensagem que não é uma opinião, enquanto "O filme é ok" 
é uma mensagem neutra.

Responda com "sim" ou "não" apenas.

Mensagem: "{text}"
"""

In [5]:
def get_opinion(model, text):
    response = ollama.chat(
        model=model,
        messages=[{
            'role': 'user',
            'content': PROMPT_TEMPLATE.format(text=text)
        }]
    )
    return response['message']['content'].lower().strip()

In [6]:
examples = [
    'Este é o novo filme do Tom Cruise',
    'Esse novo filme do Tom Cruise é incrível!',
]

for e in examples:
    p(f'[bold red]"{e}":[/bold red] {get_opinion("llama3", e)}')

"Este é o novo filme do Tom Cruise": não

"Esse novo filme do Tom Cruise é incrível!": sim

In [7]:
for data in datasets:
    df = pd.read_csv(f'../data/opinions/{data}_cachaca.csv')
    is_opinative = [
        'sim' in get_opinion("llama3", text)
        for text in df['text']
    ]
    p(f'[bold red]Total de mensagens opinativas em {data}:[/bold red] {sum(is_opinative)} / {len(is_opinative)}')
    df['is_opinative'] = is_opinative
    df.to_csv(f'../data/opinions/{data}_cachaca.csv', index=False)

# Results II

Results removing non-opinionated sentences.

In [4]:
datasets = ('facebook', 'twitter')
for data in datasets:
    df = pd.read_csv(f'../data/opinions/{data}_cachaca.csv')
    df = df[df.is_opinative == True]
    for model in models:
        y_true, y_pred = df['labels'], df[model]
        p(f'[bold red]{data}: {model}[/bold red]')
        p(classification_report(y_true, y_pred))

facebook: llama3

precision    recall  f1-score   support

    negative       0.15      0.90      0.26        29
     neutral       0.65      0.51      0.57       323
    positive       0.83      0.71      0.76       534

    accuracy                           0.64       886
   macro avg       0.54      0.71      0.53       886
weighted avg       0.74      0.64      0.68       886

facebook: mistral

precision    recall  f1-score   support

    negative       0.22      0.79      0.35        29
     neutral       0.66      0.50      0.57       323
    positive       0.77      0.78      0.78       534

    accuracy                           0.68       886
   macro avg       0.55      0.69      0.56       886
weighted avg       0.71      0.68      0.69       886

facebook: gemma2

...

precision    recall  f1-score   support

    negative       0.20      0.67      0.30        18
     neutral       0.94      0.12      0.21       380
    positive       0.21      0.90      0.35        90

    accuracy                           0.28       488
   macro avg       0.45      0.56      0.29       488
weighted avg       0.78      0.28      0.24       488

twitter: gemma2

precision    recall  f1-score   support

    negative       0.31      0.50      0.38        18
     neutral       0.91      0.16      0.27       380
    positive       0.22      0.97      0.36        90

    accuracy                           0.32       488
   macro avg       0.48      0.54      0.34       488
weighted avg       0.76      0.32      0.29       488

twitter: gpt-3.5-turbo-0125

precision    recall  f1-score   support

    negative       0.23      0.78      0.35        18
     neutral       0.89      0.25      0.39       380
    positive       0.23      0.82      0.36        90

    accuracy                           0.38       488
   macro avg       0.45      0.62      0.37       488
weighted avg       0.74      0.38      0.38       488